In [3]:
import pyspark
import pyspark
from pyspark.sql import SparkSession
import os

## DEFINE SENSITIVE VARIABLES
NESSIE_URI = "http://nessie:19120/api/v1"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password"



conf = (
    pyspark.SparkConf()
        .setAppName('app_name')
  		#packages
        .set('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.3_2.12:1.3.1,org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0,software.amazon.awssdk:bundle:2.17.178,software.amazon.awssdk:url-connection-client:2.17.178')
  		#SQL Extensions
        .set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions,org.projectnessie.spark.extensions.NessieSparkSessionExtensions')
  		#Configuring Catalog
        .set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
        .set('spark.sql.catalog.nessie.uri', NESSIE_URI)
        .set('spark.sql.catalog.nessie.ref', 'main')
        .set('spark.sql.catalog.nessie.authentication.type', 'NONE')
        .set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
        .set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse')
        .set('spark.sql.catalog.nessie.s3.endpoint', 'http://minio:9000')
        .set('spark.sql.catalog.nessie.io-impl', 'org.apache.iceberg.aws.s3.S3FileIO')
  		#MINIO CREDENTIALS
        .set('spark.hadoop.fs.s3a.access.key', MINIO_ACCESS_KEY)
        .set('spark.hadoop.fs.s3a.secret.key', MINIO_SECRET_KEY)
)

## Start Spark Session
spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("Spark Running")

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.3_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12 added as a dependency
software.amazon.awssdk#bundle added as a dependency
software.amazon.awssdk#url-connection-client added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5644be1b-78ab-4b55-8829-ad8dde0ec105;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.3_2.12;1.3.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12;0.67.0 in central
	found software.amazon.awssdk#bundle;2.17.178 in central
	found software.amazon.eventstream#eventstream;1.0.1 in central
	found software.amazon.awssdk#url-connection-client;2.17.178 in central
	found software.amazon.awssdk#utils;2.17.178 in central
	found org.reactivestreams#reactive-streams;1.0.3 in central

25/04/17 08:37:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/04/17 08:37:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/04/17 08:37:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
Spark Running


In [27]:
spark.sql("DESCRIBE TABLE EXTENDED nessie.test1").show(truncate=False)


+----------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                                                                           |comment|
+----------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|name                        |string                                                                                                                                                                              |       |
|ts                          |timestamp                                                                                 

In [17]:
table = spark.read.format("iceberg").load("nessie.test1")
table.printSchema()


root
 |-- name: string (nullable = true)
 |-- ts: timestamp (nullable = true)



In [19]:
table.schema


StructType([StructField('name', StringType(), True), StructField('ts', TimestampType(), True)])

In [10]:
spark.sql("ALTER TABLE nessie.test1 ADD COLUMN ts TIMESTAMP")

25/04/17 11:29:11 WARN BaseTransaction: Failed to load metadata for a committed snapshot, skipping clean-up


DataFrame[]

In [22]:
spark.sql("update nessie.test1 set ts=cast('2025-01-01' as date) where name='test'")

DataFrame[]

In [23]:
spark.sql("update nessie.test1 set ts=cast('2025-02-02' as date) where name<>'test'")

DataFrame[]

In [34]:
spark.sql("""
    ALTER TABLE nessie.test1 
    ADD PARTITION FIELD days(ts)
""")

AnalysisException: 
mismatched input '=' expecting {<EOF>, '.', 'AS'}
== SQL ==

    ALTER TABLE nessie.test1 
    ADD PARTITION FIELD ts_day = days(ts)


In [37]:
spark.sql("SELECT * FROM nessie.test1.partitions").show(truncate=True)


+------------+-------+------------+----------+----------------------------+--------------------------+----------------------------+--------------------------+
|   partition|spec_id|record_count|file_count|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|
+------------+-------+------------+----------+----------------------------+--------------------------+----------------------------+--------------------------+
|{2025-02-02}|      1|           1|         1|                           0|                         0|                           0|                         0|
|{2025-01-01}|      1|           1|         1|                           0|                         0|                           0|                         0|
+------------+-------+------------+----------+----------------------------+--------------------------+----------------------------+--------------------------+



In [36]:
spark.sql("""
    CALL nessie.system.rewrite_data_files(
        table => 'nessie.test1',
        options => map('min-input-files', '1')
    )
""")

DataFrame[rewritten_data_files_count: int, added_data_files_count: int, rewritten_bytes_count: bigint]